In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#importovanie potrebných knižníc
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, metrics
import numpy as np
import pandas as pd
import pickle
import os
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#načítanie datasetu
merged_csv_path = "/content/drive/MyDrive/data/new_merged.csv"
df = pd.read_csv(merged_csv_path)

class_0 = df[df['label'] == 0]
class_1 = df[df['label'] == 1]


min_class_size = min(len(class_0), len(class_1))
df_train = pd.concat([
    class_0.sample(n=min_class_size, random_state=42),
    class_1.sample(n=min_class_size, random_state=42)
])

df_test = df.drop(df_train.index)
from tensorflow.keras import layers, models, optimizers, losses, metrics
import numpy as np
import pandas as pd
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


#vyváženie datasetu
min_class_size = min(len(class_0), len(class_1))
df_balanced = pd.concat([
    class_0.sample(n=min_class_size, random_state=42),
    class_1.sample(n=min_class_size, random_state=42)
])

#rozdelenie datasetu na 80% trénovacích dát a 20% testovacích dát, stratisfikácia)
df_train, df_test = train_test_split(df_balanced, test_size=0.2, stratify=df_balanced["label"], random_state=42)

#výpis distribúcie
print(f"\n New Training Set Distribution:\n{df_train['label'].value_counts()}")
print(f" New Test Set Distribution:\n{df_test['label'].value_counts()}")

#extrakcia textov a štítkov
train_texts = df_train["clean text"].tolist()
train_labels = np.array(df_train["label"].tolist(), dtype=np.int32)

test_texts = df_test["clean text"].tolist()
test_labels = np.array(df_test["label"].tolist(), dtype=np.int32)



 New Training Set Distribution:
label
1    3605
0    3604
Name: count, dtype: int64
 New Test Set Distribution:
label
0    902
1    901
Name: count, dtype: int64


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

#nastavenie parametrov, maximalny pocet slov, dlzka sekvencie a rozmer embedding vrstvy
max_words = 10000
max_length = 100
embedding_dim = 100

#tokenizacia
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

#uloženie tokenizera
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

#konvertovanie textu na sekvencie
train_sequences = tokenizer.texts_to_sequences(train_texts)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')

test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

#výpočet váh
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(train_labels), y=train_labels)
class_weight_dict = dict(enumerate(class_weights))
print(f" Class Weights: {class_weight_dict}")

#nastavenie hyperparametrov
lstm_units = 64
dropout_rate = 0.5
learning_rate = 0.001
batch_size = 16
epochs = 10

#definovanie modelu BiGRU
model = models.Sequential([
    layers.Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_length),
    layers.Bidirectional(layers.GRU(128, return_sequences=True)),
    layers.Bidirectional(layers.GRU(64)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


#kompilácia a trénovanie modelu
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_padded, train_labels.astype(np.float32),
    validation_data=(test_padded, test_labels.astype(np.float32)),
    epochs=epochs,
    batch_size=batch_size,
    class_weight=class_weight_dict,
    verbose=1
)



In [ ]:
#testovanie modelu
#predikcie pravdepodobností
y_pred_probs = model.predict(test_padded)
# Prah 0.5
y_pred = (y_pred_probs > 0.5).astype("int32")

#výpočet metrík
acc = accuracy_score(test_labels, y_pred)
prec = precision_score(test_labels, y_pred)
rec = recall_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)

#výpis
print("\n Výsledky na testovacej množine:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")


57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step

 Výsledky na testovacej množine:
Accuracy:  0.6151
Precision: 0.6151
Recall:    0.6138
F1 Score:  0.6144


In [ ]:
#výpis rozloženia výstupov
y_pred_probs = model.predict(test_padded)
print("Min:", np.min(y_pred_probs), "Max:", np.max(y_pred_probs), "Priemer:", np.mean(y_pred_probs))

y_pred = (y_pred_probs > 0.3).astype("int32")

print("Predicted label counts:", np.unique(y_pred, return_counts=True))


57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step
Min: 6.9117135e-07 Max: 0.99999607 Priemer: 0.49549776
Predicted label counts: (array([0, 1]), array([816, 987], dtype=int64))
